# Preprocessing
We preprocess models in this file through the following procedure:
1. Load the model through Open3D
2. Normalize the model such that it fits inside a 32x32x32 voxel grid:
    - Normalize the bounding box to $ [-1.0]^3 $ to $ [1.0]^3 $
    - Voxelize using Open3D
3. Convert the model to an occupancy map and store to a file in human-readable format.

In [1]:
import torch
import numpy as np
import open3d as o3d
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Warning)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Read mesh
mesh = o3d.io.read_triangle_mesh("../data/obj_data/shapenet/1021a0914a7207aff927ed529ad90a11/models/model_normalized.obj")
mesh

TriangleMesh with 7526 points and 8448 triangles.

In [3]:
def normalize(mesh):
    '''
    Normalize mesh in-place.
    '''
    center = mesh.get_center()
    mesh.translate(-center)
    extent = mesh.get_axis_aligned_bounding_box().get_max_extent()
    mesh.scale(2.0 / extent, [0, 0, 0])


AABB = mesh.get_axis_aligned_bounding_box()
print(f"Origin {AABB},\n\t max extent={AABB.get_max_extent()}")

normalize(mesh)

AABB = mesh.get_axis_aligned_bounding_box()
print(f"New {AABB},\n\t max extent={AABB.get_max_extent()}")

Origin AxisAlignedBoundingBox: min: (-0.359457, -0.065165, -0.299895), max: (0.360894, 0.144524, 0.36126),
	 max extent=0.720350980758667
New AxisAlignedBoundingBox: min: (-1.00001, -0.177738, -0.848785), max: (0.999988, 0.404448, 0.986862),
	 max extent=2.0


In [4]:
o3d.visualization.draw_geometries([mesh])

The following part is related to voxelization.

In [5]:
pcd = mesh.sample_points_uniformly(number_of_points=1000)
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud_within_bounds(pcd, voxel_size=float(2/32 + 0.001), min_bound=[-1., -1., -1.], max_bound=[1., 1., 1.])
omap = np.zeros((32, 32, 32), dtype=int)

for voxel in voxel_grid.get_voxels():
    # print(voxel.grid_index)
    omap[tuple(voxel.grid_index)] = 1
o3d.visualization.draw_geometries([voxel_grid])

In [6]:
bounds = np.linspace(-1.0, 1.0, 33, dtype=np.float32)
primitive_coords = (bounds[1:] + bounds[:-1]) / 2
coord_grid = np.array(np.meshgrid(primitive_coords, primitive_coords, primitive_coords, indexing='ij'))

x, y, z = coord_grid
x = x.reshape(-1, 1)
y = y.reshape(-1, 1)
z = z.reshape(-1, 1)
coords = np.concatenate([x, y, z], axis=1)
coords

array([[-0.96875, -0.96875, -0.96875],
       [-0.96875, -0.96875, -0.90625],
       [-0.96875, -0.96875, -0.84375],
       ...,
       [ 0.96875,  0.96875,  0.84375],
       [ 0.96875,  0.96875,  0.90625],
       [ 0.96875,  0.96875,  0.96875]], dtype=float32)

In [7]:
# First, transform the coords to target space
# Then compute the closest points in the target space
# After computing the closest points, use the offset to convert these points to STD space
# Sample the points inside the target space

In [8]:
voxels = voxel_grid.get_voxels()
base = np.array([1024, 32, 1])
n_voxels = len(voxels)

vcenters = np.array([voxel_grid.get_voxel_center_coordinate(voxel.grid_index) for voxel in voxels])
std_centers = np.array([coords[np.dot(voxel.grid_index, base)] for voxel in voxels])
print(f'vcenters.shape={vcenters.shape}')
print(f'std_centers.shape={std_centers.shape}')

print(f'standard_width={2/32}')
offset_vec = np.mean(vcenters - std_centers, axis=0)

print(f'offset vector={offset_vec}')
coords_1 = coords[:] + offset_vec

transformed_centers = std_centers[:] + offset_vec

for i in range(n_voxels):
    grid_center = vcenters[i]
    transformed_center = transformed_centers[i]
    print(f'{i}\t{grid_center}\t\t{transformed_center}')

vcenters.shape=(401, 3)
std_centers.shape=(401, 3)
standard_width=0.0625
offset vector=[0.01592643 0.01591646 0.01747506]
0	[-0.14275  0.04775  0.49225]		[-0.14032357  0.04716646  0.48622506]
1	[-0.01575 -0.07925 -0.01575]		[-0.01532357 -0.07783354 -0.01377494]
2	[-0.07925  0.04775  0.49225]		[-0.07782357  0.04716646  0.48622506]
3	[-0.14275  0.04775  0.36525]		[-0.14032357  0.04716646  0.36122506]
4	[-0.01575 -0.07925 -0.14275]		[-0.01532357 -0.07783354 -0.13877494]
5	[ 0.49225 -0.07925  0.04775]		[ 0.48467643 -0.07783354  0.04872506]
6	[-0.14275  0.17475  0.68275]		[-0.14032357  0.17216646  0.67372506]
7	[-0.33325  0.17475  0.80975]		[-0.32782357  0.17216646  0.79872506]
8	[ 0.55575 -0.01575  0.11125]		[ 0.54717643 -0.01533354  0.11122506]
9	[-0.14275  0.17475  0.74625]		[-0.14032357  0.17216646  0.73622506]
10	[-0.07925  0.11125  0.74625]		[-0.07782357  0.10966646  0.73622506]
11	[-0.14275  0.11125  0.74625]		[-0.14032357  0.10966646  0.73622506]
12	[-0.07925  0.11125  0.68275]		[-0

In [ ]:
print(mesh.get_center())
print(voxel_grid.get_center())

[ 7.36684461e-16  2.09704696e-16 -3.73166080e-17]
[-0.0018495   0.00794403  0.11630473]


In [ ]:
test_idx = np.array([[0], [0], [1]])
print(test_idx.shape)
coords[test_idx]

(3, 1)


array([[[[[-1.        , -1.        , -1.        ],
          [-1.        , -1.        , -0.9354839 ],
          [-1.        , -1.        , -0.87096775],
          ...,
          [-1.        , -1.        ,  0.87096775],
          [-1.        , -1.        ,  0.9354839 ],
          [-1.        , -1.        ,  1.        ]],

         [[-1.        , -1.        , -0.9354839 ],
          [-1.        , -0.9354839 , -0.9354839 ],
          [-1.        , -0.9354839 , -0.87096775],
          ...,
          [-1.        , -0.9354839 ,  0.87096775],
          [-1.        , -0.9354839 ,  0.9354839 ],
          [-1.        , -0.9354839 ,  1.        ]],

         [[-1.        , -1.        , -0.87096775],
          [-1.        , -0.9354839 , -0.87096775],
          [-1.        , -0.87096775, -0.87096775],
          ...,
          [-1.        , -0.87096775,  0.87096775],
          [-1.        , -0.87096775,  0.9354839 ],
          [-1.        , -0.87096775,  1.        ]],

         ...,

         [[-1.  

In [11]:
def compute_closest_points_to_grids(mesh, grid_centers):
    '''
    Precompute, for each grid, the closest point on the mesh to that specific grid. 
    
    Return a tensor size of 32x32x32x3.
    
    The mesh itself must has already been normalized.
    '''
    mesh0 = o3d.t.geometry.TriangleMesh.from_legacy(mesh)
    scene = o3d.t.geometry.RaycastingScene()
    _ = scene.add_triangles(mesh0)
    ans = scene.compute_closest_points(np.array(grid_centers, dtype=np.float32))
    return ans['points'].numpy()

grid_points = compute_closest_points_to_grids(mesh, transformed_centers)
grid_points

[Open3D WARNING] Legacy mesh has more than 1 material which is not supported by Tensor-based meshes.


array([[-1.1712979e-01,  3.3432536e-02,  4.8274779e-01],
       [-1.5323547e-02, -1.0343254e-01, -1.3774887e-02],
       [-7.7631064e-02,  3.3856768e-02,  4.8551878e-01],
       ...,
       [ 2.9672605e-01, -7.0225403e-02,  1.1077542e-01],
       [-1.5323142e-02, -1.0343254e-01, -3.8877538e-01],
       [-5.7420146e-04,  2.9170057e-01,  7.4210799e-01]], dtype=float32)